# Explore the distibution of birth years

In [ ]:
import sqlite3 as sql
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px

## Get the birth dates (or years) from the SQLite database


In [ ]:
### Define the path to the file (a string in a variable 'db_file')
db_file = '../../data/peace_import_auto.db'

In [ ]:
### Créer une connexion vers la base de données
# Attention: si la base de données n'existe pas, elle est créée à l'endroit indiqué.
# Ce qui est une manière de la créer si souhaité
try:
    cn = sql.connect(db_file)
except Exception as e:
    print(e)    
# Le contenu de la variable: connexion
# cn

In [ ]:
### La requête à effectuer
# Préparée généralement directement sur la BD SQLite
# Elle produit la liste des années de naissance,
# une année par individu
q1 = """
SELECT birth_year 
FROM person p
order by birth_year 
"""

In [ ]:
### Création du conteneur du résultat de la requête
try:
    cur = cn.cursor()
except Exception as e:
    print(e)    

In [ ]:
### Exécuter la requête et récupérer le résultat
cur.execute(q1)
# La méthode 'fetchall' permet de récupérer toutes les lignes
r = cur.fetchall()
cn.close()

In [ ]:
### Inspecter le résultat:
# longueur, type du résultat et premières lignes du résultat
# On observe qu'il y a des trous et des répétitions:
# plusieurs personnes on la même année de naissance
print(len(r), '\n-----\n', type(r),  '\n-----\n', r[:5])

## Python control structures

Read and treat each element (e) of a list 
* A [Python list](https://datacarpentry.github.io/python-socialsci/02-basics.html#the-list)
* [*for* loop](https://datacarpentry.github.io/python-socialsci/03-control-structures.html#the-for-loop) (Datacarpentry tutorial)
* [*while* loop](https://datacarpentry.github.io/python-socialsci/03-control-structures.html#the-while-loop)
* [*if* statement](https://datacarpentry.github.io/python-socialsci/03-control-structures.html#the-if-statement-and-variants)

In [ ]:
### Result of SQL query: list of tuples
# five first elements
print(r[:5], '\n---')
# access the values
print(r[0], r[1], r[1][0])

In [ ]:
# 'for' loop on the first five elements
for e in r[:5]:
    print(e[0])

In [ ]:
# 'while' loop on the first five elements
i = 0
while i < 5 :
    print(r[i][0])
    i += 1

In [ ]:
# 'while' loop on the first 20 elements
# adding an if condition
i = 0
while i < 20 :
    # modulus: returns the reminder
    if r[i][0] % 5 == 0:
        print(r[i][0])
        i += 1
    else:
        i += 1    


## Prepare the data to be plotted

In [ ]:
### Create a new list of values, not tuples
# create an empty list named 'lr'
lr = []

# add (append) each element (the first part of it) to the list 'lr'.
for e in r:
    lr.append(e[0])

print(lr[:5])

In [ ]:
### Test if all list elements are of type 'integer'
# Doc.: https://stackoverflow.com/questions/6009589/how-to-test-if-every-item-in-a-list-of-type-int
all(type(item) is int for item in lr)

In [ ]:
### Extract the year from the date and convert value from string to integer
dt = '1457-08-02'
a = int(dt[:4]) ; a

In [ ]:
### Apply to the whole list:
# first, convert all values to string
# second, take only the first four characteurs, i.e. the year
# then convert to integer
lr = []
for e in r:
    lr.append(int((str(e[0]))[:4]))


print(lr[:5])    

In [ ]:
### Create a Pandas series (a particular list object)
slr = pd.Series(lr, name='Année')
print(slr[:2])

In [ ]:
slrg = slr.groupby(by=slr).size()
slrg.name='Année'
slrg.tail()

In [ ]:
### Group by year and plot
# Beware : a lot of years are missing !
# Avoid this approach !

fig, ax = plt.subplots()



slrg.plot(kind='bar', figsize=(20, 5))

## specific xticks labels customisation
every_nth = 5
for n, label in enumerate(ax.xaxis.get_ticklabels()):
    if n % every_nth != 0:
        label.set_visible(False)
plt.xticks(rotation=30)        
plt.show()

In [ ]:

print (lr[:3], lr[-7:], 'Min: ' + str(min(lr)), 'Max: ' + str(max(lr)))


In [ ]:
y_l = list(range(min(lr), max(lr) + 1, 1))
print(len(y_l),y_l[:7])

In [ ]:
### Count how many births there are per year
# Parse a list inside another list: can be slow for long lists !
# There are other methods, here we just want to illustrate the idea
y_r = []
for a in y_l:
    # initialize the counter for each year
    n = 0    
    for v in lr:        
        if v == a:
            n += 1
        else:
            pass    
    y_r.append([a,n])
print(y_r[:10])  

In [ ]:
### Create a Pandas dataframe (a kind of SQL table)
df_y_r = pd.DataFrame(y_r, columns=['year', 'number'])
df_y_r.tail(10)

In [ ]:
### Group by year and plot


fig, ax = plt.subplots()

df_y_r.plot(x='year', y='number',kind='bar', figsize=(20, 5), ax=ax)

## specific xticks labels customisation
every_nth = 10
for n, label in enumerate(ax.xaxis.get_ticklabels()):
    if n % every_nth != 0:
        label.set_visible(False)
ax.tick_params(axis='x', labelsize=10,  rotation=30)
plt.show()

## Plot with the Plotly library

In [ ]:
# The original series
print(slrg.head(), slrg.tail())

In [ ]:
### Use a different plotting library: Plotly Express
# Observe the default behaviour of the library

fig = px.bar(slrg)
fig.update_traces(width=2)
fig.show()

### Codage par période de vingt-cinq ans

In [ ]:
type(slrg), slrg.tail()

In [ ]:
df_slrg = slrg.reset_index(name='Effectif')
df_slrg.tail(10)

In [ ]:
### Créer une liste d'années pour séparer en périodes de 25 ans
# noter que la dernière année sera exclue, 
# elle donc doit être supérieure à la valeur maximale 
l_25 = list(range(1351, 2002, 25))
print(l_25[:5],l_25[-5:])

In [ ]:
### fonction pd.cut : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html
# On ajoute une nouvelle colonne qui contient la période sur la base de la liste précédente
# et de la valeur de l'année

df_slrg['Périodes'] = pd.cut(df_slrg.Année, l_25, right=False)

### Transformer le code ajouté pour qu'il soit plus lisible
# noter qu'on a arrondi les valeurs
df_slrg['Périodes'] = df_slrg['Périodes'].apply(lambda x : str(int(x.left))+'-'+ str(int(x.right)-1))

# Inspection
df_slrg.head(3)

In [ ]:
### compter les naissances par périodes de 25 ans
df_slrg[['Périodes','Effectif']].groupby(by='Périodes').sum().head()

In [ ]:
g_df = df_slrg[['Périodes','Effectif']].groupby(by='Périodes').sum()

ax = g_df.plot(kind='bar',rot=60, fontsize=8, figsize=(16,8))
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() + 0.08 , p.get_height() * 1.02))
plt.ylabel('Effectif')
plt.xlabel('Périodes')
plt.title('Nombre de naissances par périodes de ving-cinq ans')
plt.savefig('pictures/birth_years_plt_20241208.png')
plt.show()

In [ ]:
### Use a different plotting library


fig = px.bar(g_df, 
             # https://plotly.com/python/figure-labels/
             title='Effectifs des naissances par périodes de 25 ans',
             labels={'periodes':'Période',
                     'value':'Effectif',
                     'variable':'Effectif'},
                 width=1200, height=600)
fig.write_html("pictures/birth_years_20241208.html")
fig.write_image("pictures/birth_years_20241208.png")
fig.show()